## 0. Setup

In [ ]:
# Installing ultralytics
!pip install ultralytics torch torchvision
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.57 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.1/78.2 GB disk)


In [ ]:
# mount google drive to share data with colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Select YOLOv8 🚀 logger {run: 'auto'}
logger = 'Comet' #@param ['Comet', 'TensorBoard']

if logger == 'Comet':
  %pip install -q comet_ml
  import comet_ml; comet_ml.login()
elif logger == 'TensorBoard':
  %load_ext tensorboard
  %tensorboard --logdir .

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.5/679.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 979.1/979.1 kB 46.0 MB/s eta 0:00:00


## 0. Check
Here we check three functionalities on the 8 coco8 images

In [ ]:
# Training YOLOv8s on COCO8 for 3 epochs to check functionality
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/Zywalewski_2024_SSP/coco8/coco8.yaml epochs=5 imgsz=640

100% 21.5M/21.5M [00:00<00:00, 269MB/s]
Ultralytics YOLOv8.2.38 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/Zywalewski_2024_SSP/coco8/coco8.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, s

In [ ]:
# Validate YOLOv8s on COCO8 val
!yolo task=detect mode=val model=yolov8s.pt data=/content/drive/MyDrive/Zywalewski_2024_SSP/coco8/coco8.yaml

Ultralytics YOLOv8.2.38 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s summary (fused): 168 layers, 11156544 parameters, 0 gradients, 28.6 GFLOPs
val: Scanning /content/drive/MyDrive/Zywalewski_2024_SSP/coco8/labels/val.cache... 4 images, 0 backgrounds, 0 corrupt: 100% 4/4 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 1/1 [00:00<00:00,  1.15it/s]
                   all          4         17       0.87       0.92      0.947      0.719
                person          3         10      0.855       0.59      0.707      0.379
                   dog          1          1      0.886          1      0.995      0.796
                 horse          1          2      0.794          1      0.995        0.8
              elephant          1          2          1      0.931      0.995       0.55
              umbrella          1          1      0.686          1      0.995      0.895
          potted plant     

In [ ]:
# Test YOLOv8n on COCO8 val
!yolo task=detect mode=predict model=yolov8s.pt conf=0.25 imgsz=1280 line_thickness=1 source='https://ultralytics.com/images/zidane.jpg'

WARNING ⚠️ 'line_thickness' is deprecated and will be removed in 'ultralytics 8.399999999999999' in the future. Please use 'line_width' instead.
Ultralytics YOLOv8.2.38 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s summary (fused): 168 layers, 11156544 parameters, 0 gradients, 28.6 GFLOPs

100% 165k/165k [00:00<00:00, 8.59MB/s]
image 1/1 /content/zidane.jpg: 736x1280 2 persons, 1 tie, 180.1ms
Speed: 21.9ms preprocess, 180.1ms inference, 1073.4ms postprocess per image at shape (1, 3, 736, 1280)
Results saved to runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


## 1. Train
Training a YOLOv8 model with the provided train images and their bounding boxes. First, I train for 2 epochs and see if the code runs without error. Then I train with more epochs and store the model weights.

In [ ]:
# Building a YOLOv8 small model from YAML and start training from scratch for 2 epochs
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/Zywalewski_2024_SSP/baseball/baseball.yaml epochs=2 imgsz=640

Ultralytics YOLOv8.2.38 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/Zywalewski_2024_SSP/baseball/baseball.yaml, epochs=2, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, 

In [ ]:
# Training again with 600 epochs once the above block runs error free
# After training is complete, I download the best.pt file
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/Zywalewski_2024_SSP/baseball/baseball.yaml epochs=600 imgsz=640

100% 21.5M/21.5M [00:00<00:00, 450MB/s]
Ultralytics YOLOv8.2.57 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/Zywalewski_2024_SSP/baseball/baseball.yaml, epochs=600, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=

## 2. Detect

In [ ]:
# Runingn a detection using a trained small model
!yolo task=detect mode=predict model=/content/drive/MyDrive/Zywalewski_2024_SSP/baseball/best.pt conf=0.25 imgsz=640 line_thickness=1 source=/content/drive/MyDrive/Zywalewski_2024_SSP/baseball/images/test

WARNING ⚠️ 'line_thickness' is deprecated and will be removed in in the future. Please use 'line_width' instead.
Ultralytics YOLOv8.2.57 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs

image 1/9 /content/drive/MyDrive/Zywalewski_2024_SSP/baseball/images/test/FA16_frame012_jpg.rf.a4eea2570c977d7b6760238b95adfe81.jpg: 384x640 1 Bat, 114.0ms
image 2/9 /content/drive/MyDrive/Zywalewski_2024_SSP/baseball/images/test/FA16_frame033_jpg.rf.7030ed538ca502ca46e939c35bab2061.jpg: 384x640 1 Baseball, 1 Bat, 11.4ms
image 3/9 /content/drive/MyDrive/Zywalewski_2024_SSP/baseball/images/test/FA16_frame036_jpg.rf.b29c99c4ab8f65999a407945d794c546.jpg: 384x640 1 Baseball, 1 Bat, 10.7ms
image 4/9 /content/drive/MyDrive/Zywalewski_2024_SSP/baseball/images/test/FA5_frame059_jpg.rf.3c2e6e274e4a90f46fecb630a936db75.jpg: 384x640 1 Baseball, 1 Bat, 10.8ms
image 5/9 /content/drive/MyDrive/Zywalewski_2024_SSP/basebal

In [ ]:
# Run a detection on a single video
# PATH = '/content/drive/MyDrive/Zywalewski_2024_SSP'
!yolo task=detect mode=predict model=/content/drive/MyDrive/Zywalewski_2024_SSP/baseball/best.pt conf=0.25 imgsz=640 line_width=1 source=/content/drive/MyDrive/Zywalewski_2024_SSP/videos/videos_used_for_image_extraction/side_angle_2.mp4

Ultralytics YOLOv8.2.57 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs

video 1/1 (frame 1/94) /content/drive/MyDrive/Zywalewski_2024_SSP/videos/videos_used_for_image_extraction/side_angle_2.mp4: 384x640 1 Bat, 150.2ms
video 1/1 (frame 2/94) /content/drive/MyDrive/Zywalewski_2024_SSP/videos/videos_used_for_image_extraction/side_angle_2.mp4: 384x640 1 Bat, 10.8ms
video 1/1 (frame 3/94) /content/drive/MyDrive/Zywalewski_2024_SSP/videos/videos_used_for_image_extraction/side_angle_2.mp4: 384x640 1 Bat, 10.7ms
video 1/1 (frame 4/94) /content/drive/MyDrive/Zywalewski_2024_SSP/videos/videos_used_for_image_extraction/side_angle_2.mp4: 384x640 1 Bat, 11.5ms
video 1/1 (frame 5/94) /content/drive/MyDrive/Zywalewski_2024_SSP/videos/videos_used_for_image_extraction/side_angle_2.mp4: 384x640 1 Bat, 10.7ms
video 1/1 (frame 6/94) /content/drive/MyDrive/Zywalewski_2024_SSP/videos/videos_used_for_image_ext

In [ ]:
# Run a detection on all edited videos
!yolo task=detect mode=predict model=/content/drive/MyDrive/Zywalewski_2024_SSP/baseball/best.pt conf=0.25 imgsz=640 line_width=1 source=/content/drive/MyDrive/Zywalewski_2024_SSP/videos/all_edited_videos

Streaming output truncated to the last 5000 lines.

video 1/56 (frame 1/55) /content/drive/MyDrive/Zywalewski_2024_SSP/videos/all_edited_videos/front_angle_1.mp4: 384x640 (no detections), 110.2ms
video 1/56 (frame 2/55) /content/drive/MyDrive/Zywalewski_2024_SSP/videos/all_edited_videos/front_angle_1.mp4: 384x640 (no detections), 10.7ms
video 1/56 (frame 3/55) /content/drive/MyDrive/Zywalewski_2024_SSP/videos/all_edited_videos/front_angle_1.mp4: 384x640 (no detections), 10.7ms
video 1/56 (frame 4/55) /content/drive/MyDrive/Zywalewski_2024_SSP/videos/all_edited_videos/front_angle_1.mp4: 384x640 1 Bat, 10.7ms
video 1/56 (frame 5/55) /content/drive/MyDrive/Zywalewski_2024_SSP/videos/all_edited_videos/front_angle_1.mp4: 384x640 (no detections), 11.5ms
video 1/56 (frame 6/55) /content/drive/MyDrive/Zywalewski_2024_SSP/videos/all_edited_videos/front_angle_1.mp4: 384x640 1 Bat, 10.7ms
video 1/56 (frame 7/55) /content/drive/MyDrive/Zywalewski_2024_SSP/videos/all_edited_videos/front_angle_1.mp4

In [ ]:
# Copy the folder to your Google Drive
!cp -r /content/runs/detect/predict3 /content/drive/MyDrive/Zywalewski_2024_SSP/Results

'1st dataset- 250 epochs results of baseball training'	'Code Attempts.gdoc'
'1st dataset- 500 epochs results of baseball training'	'final baseball dataset- 600 epochs'
'2nd dataset- 350 epochs results of baseball training'	 predict3


In [ ]:
# Running a tracker on a video
# PATH = '/content/drive/MyDrive/Zywalewski_2024_SSP'
!yolo mode=track model=/content/drive/MyDrive/Zywalewski_2024_SSP/baseball/best.pt source=/content/drive/MyDrive/Zywalewski_2024_SSP/videos/videos_used_for_image_extraction/side_angle_2.mp4 conf=0.25 show

requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.3 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 7.1s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Ultralytics YOLOv8.2.53 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

video 1/1 (frame 1/94) /content/drive/MyDrive/Zywalewski_2024_SSP/videos/videos_used_for_image_extraction/side_angle_2.mp4: 384x640 1 Bat, 108.5ms
video 1/1 (frame 2/94) /content/drive/MyDrive/Zywalewski_2024_SSP/videos/videos_used_for_image_extraction/side_angle_2.mp4: 384x640 1 Bat, 13.8ms
video 1/1 (frame 3/94) /content/drive/MyDrive/Zywalewski_2024_SSP/videos/videos_used_for_image_extraction/side_angle_2.mp4: 384x640 1 Bat,

## 2+. Extra Images
Since the model did not perfroming well-enough, I needed to add more images for better training. The boudnign box process was repeated for the new images and added to the dataset.

The training process was completed again, using the block of code above.

In [ ]:
import imageio.v3 as iio
import cv2

In [ ]:
## Generating frame by frame images of input videos
for idx, frame in enumerate(iio.imiter("/content/drive/MyDrive/Zywalewski_2024_SSP/videos/videos_used_for_image_extraction/side_angle_21.mp4")):
    iio.imwrite(f"/content/drive/MyDrive/Zywalewski_2024_SSP/images/frames/side_angle_21/frame{idx:03d}.jpg", frame)

# 3. Estimation
After going back to the training step, I had a well-trained object detection model. I could now use this to track the baseball in the videos and calculate exit velocity.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics opencv-python-headless

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [ ]:
video_path = '/content/drive/My Drive/Zywalewski_2024_SSP/videos/side_angle_after_contact/side_angle_5.mp4' # Update depending on which video

In [ ]:
import cv2

from ultralytics import YOLO

model_path = '/content/drive/My Drive/Zywalewski_2024_SSP/baseball/best.pt'
model = YOLO(model_path)

cap = cv2.VideoCapture(video_path)

frame_rate = cap.get(cv2.CAP_PROP_FPS)
positions = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Get predictions
    results = model(frame)

    # Extract coordinates of the ball
    for pred in results[0].boxes.data:
        if int(pred[-1]) == 0:  # Assuming class 0 is the ball
            x_center = (pred[0] + pred[2]) / 2
            y_center = (pred[1] + pred[3]) / 2
            positions.append((x_center.item(), y_center.item()))
            break

cap.release()


0: 384x640 1 Baseball, 1 Bat, 11.6ms
Speed: 2.0ms preprocess, 11.6ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Baseball, 1 Bat, 23.3ms
Speed: 1.7ms preprocess, 23.3ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Baseball, 1 Bat, 15.5ms
Speed: 1.7ms preprocess, 15.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Baseball, 1 Bat, 17.0ms
Speed: 3.7ms preprocess, 17.0ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Baseball, 1 Bat, 11.6ms
Speed: 1.7ms preprocess, 11.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Bat, 13.2ms
Speed: 3.7ms preprocess, 13.2ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)


In [ ]:
import numpy as np

def calculate_speed(positions, frame_rate):
    speeds = []
    for i in range(1, len(positions)):
        # Calculate distance in pixels
        dist = np.linalg.norm(np.array(positions[i]) - np.array(positions[i - 1]))

        # Calculate speed (distance per second)
        speed = dist * frame_rate  # assuming positions are in pixels and frame_rate is frames per second
        speeds.append(speed)

    return speeds

speeds = calculate_speed(positions, frame_rate)
print(f'Average speed: {np.mean(speeds)} pixels/second')

estimated_miles_per_pixel = 0.00002994541145
print(f'Exit Velocity: {(np.mean(speeds))*estimated_miles_per_pixel*3600} mph')

Average speed: 1045.5709155814557 pixels/second
Exit Velocity: 112.71618456206365 mph
